In [138]:
#import libraries 
import pandas as pd
import numpy as np
from tqdm import tqdm
from os.path import exists
import os

from itertools import product

In [139]:
#root path
path = '/nfsmount/majid/multimodal/Emotion_recognition_T/dataset/'

In [140]:
# options and assumptions
participant_list = ['H','I','J','K','L','M','N','P','Q','S']
video_length = 1800 #frames(30 fps) = 1 minute
sliding = 1800 #tumbling window
#Dataset
df = pd.DataFrame(columns=['video','start_frame', 'end_frame','biometric','start_row','end_row','stress'])

In [141]:
def annotate(participant,task,minute):
    #checks if the file exists
    if (exists(f'{os.path.join(path, participant)}/V{task}_facecroppad.npy')) and (exists(f'{os.path.join(path, participant)}/T{task}.csv')):
        #extract the stress level from the biometric files 
        bio_df = pd.read_csv(f'{os.path.join(path, participant)}/T{task}.csv')['stress']

        bio = bio_df.iloc[1+minute * 240]

        #combine the data information as an array
        return [f'{os.path.join(path, participant)}/V{task}_facecroppad.npy',
              minute*video_length, (minute)*video_length + sliding,
              f'{os.path.join(path, participant)}/T{task}.csv',
              minute * 240,
              (minute+1) * 240 , bio]

for vals in product(participant_list, range(1,9), range(9)):
    df.loc[len(df)] = annotate(*vals)

/tmp/ipykernel_83818/3585970789.py:17: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df.loc[len(df)] = annotate(*vals)
/tmp/ipykernel_83818/3585970789.py:17: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df.loc[len(df)] = annotate(*vals)
/tmp/ipykernel_83818/3585970789.py:17: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df.loc[len(df)] = annotate(*vals)
/tmp/ipykernel_83818/3585970789.py:17: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df.loc[len(df)] = annotate(*vals)
/tmp/ipykernel_83818/3585970789.py:17: FutureWarning: Th

In [146]:
#train_test_split
train = df.sample(frac = 0.6, random_state= 42)
test = df.drop(train.index)
validation = test.sample(frac = 0.5,random_state= 42)
test = test.drop(validation.index)
train['label'] = 'train'
test['label'] = 'test'
validation['label'] = 'validation'

df = pd.concat([train, validation, test], axis=0)
#save the dataset
df.to_csv('annotations.csv', index=False)

In [143]:
#check the train test split
df.label.value_counts()

train         432
validation    144
test          144
Name: label, dtype: int64

In [147]:
df.to_csv('annotations.csv', index=False)